<a href="https://colab.research.google.com/github/mjgpinheiro/AI-Strategies-StockMarket/blob/main/UO_Prediction_BTCUSDT_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import requests
from scipy.stats import linregress
from datetime import datetime
from time import sleep
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Step 1: Fetching BTC/USDT historical data from CoinGecko for 2 days
def fetch_coingecko_data():
    try:
        print("Fetching BTC/USDT data from CoinGecko...")
        logging.info("Fetching BTC/USDT data from CoinGecko...")

        # CoinGecko endpoint for OHLC data (past 2 days)
        url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=2"

        response = requests.get(url)
        data = response.json()

        # Check for 'prices' and 'total_volumes' keys
        if 'prices' not in data or 'total_volumes' not in data:
            raise ValueError("Unexpected data format: 'prices' or 'total_volumes' missing from response.")

        # Extracting the data
        prices = data['prices']
        volumes = data['total_volumes']

        # Create a DataFrame from fetched data
        df = pd.DataFrame(prices, columns=['timestamp', 'Close'])
        df['Volume'] = [v[1] for v in volumes]
        df['CloseTime'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.drop(columns='timestamp', inplace=True)

        print("Data fetched successfully from CoinGecko.")
        logging.info("Data fetched successfully from CoinGecko.")
        return df[['CloseTime', 'Close', 'Volume']]
    except Exception as e:
        print(f"Error fetching data: {e}")
        logging.error(f"Error fetching data: {e}")
        return None

# Step 2: Function to estimate parameters using the Modified OU Model
def estimate_ou_parameters(data):
    try:
        data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
        data = data.dropna()

        # Prepare data for regression (OU process treated as AR(1) model)
        X_t_minus_1 = data['Close'].shift(1).dropna().values.flatten()
        X_t = data['Close'][1:].values.flatten()

        regression_result = linregress(X_t_minus_1, X_t)

        slope = regression_result.slope
        intercept = regression_result.intercept

        # Calculating OU parameters
        theta = 1 - slope
        mu = intercept / (1 - slope)
        sigma = data['Log_Returns'].std()

        print(f"Estimated Parameters: Theta={theta}, Mu={mu}, Sigma={sigma}")
        logging.info(f"Estimated Parameters: Theta={theta}, Mu={mu}, Sigma={sigma}")
        return theta, mu, sigma
    except Exception as e:
        print(f"Error during parameter estimation: {e}")
        logging.error(f"Error during parameter estimation: {e}")
        return None, None, None

# Step 3: Adjust VAH and VAL with Fixed Buffer Around Mu
def adjust_value_areas_with_buffer(mu, buffer=50):
    vah = mu + buffer
    val = mu - buffer

    print(f"Adjusted VAH: {vah}, Adjusted VAL: {val}")
    logging.info(f"Adjusted VAH: {vah}, Adjusted VAL: {val}")
    return val, vah

# Step 4: Calculate Expected Exit Time using the OU Model
def calculate_expected_exit_time(mu, theta, sigma, val, vah):
    a = val
    b = vah

    print(f"Calculating exit time with Mu: {mu}, VAL: {a}, VAH: {b}")
    logging.info(f"Calculating exit time with Mu: {mu}, VAL: {a}, VAH: {b}")

    try:
        # Ensure VAL < Mu < VAH
        if not (a < mu < b):
            print("Mu is outside the VAL-VAH range. Adjusting values.")
            logging.warning("Mu is outside the VAL-VAH range. Adjusting values.")
            return "More than 1 hour"

        # Exit time approximation formula
        term1 = (1 / theta) * np.log((b - mu) / (mu - a))
        term2 = (sigma ** 2) / (2 * (theta ** 3)) * ((1 / (b - mu) ** 2) + (1 / (mu - a) ** 2))
        exit_time = term1 + term2

        # Categorize exit times for better predictions
        if exit_time < 0.5:  # Less than 30 minutes
            return "Less than 30 minutes"
        elif exit_time < 1:  # Less than 1 hour
            return "Within 1 hour"
        else:  # More than 1 hour
            return "More than 1 hour"
    except Exception as e:
        print(f"Error calculating expected exit time: {e}")
        logging.error(f"Error calculating expected exit time: {e}")
        return "Unknown"

# Step 5: Real-time prediction loop using the Modified OU Model
def real_time_trading(sleep_interval=300):
    while True:
        try:
            # Fetch latest data from CoinGecko
            data = fetch_coingecko_data()
            if data is None:
                print("Failed to fetch data. Retrying after sleep interval...")
                logging.warning("Failed to fetch data. Retrying after sleep interval...")
                sleep(sleep_interval)
                continue

            # Estimate OU parameters
            theta, mu, sigma = estimate_ou_parameters(data)
            if theta is None or mu is None or sigma is None:
                print("Skipping prediction due to regression error.")
                logging.info("Skipping prediction due to regression error.")
                sleep(sleep_interval)
                continue

            # Adjust VAH and VAL using a buffer around Mu
            val, vah = adjust_value_areas_with_buffer(mu, buffer=50)

            # Calculate expected exit time
            exit_time = calculate_expected_exit_time(mu, theta, sigma, val, vah)
            current_price = float(data['Close'].iloc[-1].item())

            # Provide clear trading advice based on the current price position
            if exit_time == "Less than 30 minutes":
                if current_price > vah:
                    print("Advice: BUY - Price is likely to break UP from the range.")
                    logging.info("Advice: BUY - Price is likely to break UP from the range.")
                elif current_price < val:
                    print("Advice: SELL - Price is likely to break DOWN from the range.")
                    logging.info("Advice: SELL - Price is likely to break DOWN from the range.")
                else:
                    print("Advice: WATCH - Immediate action likely, but direction unclear.")
                    logging.info("Advice: WATCH - Immediate action likely, but direction unclear.")
            elif exit_time == "Within 1 hour":
                if current_price > vah:
                    print("Advice: BUY - Expected upward movement within an hour.")
                    logging.info("Advice: BUY - Expected upward movement within an hour.")
                elif current_price < val:
                    print("Advice: SELL - Expected downward movement within an hour.")
                    logging.info("Advice: SELL - Expected downward movement within an hour.")
                else:
                    print("Advice: HOLD - Expect movement but stay alert.")
                    logging.info("Advice: HOLD - Expect movement but stay alert.")
            else:
                print("Advice: HOLD - Price likely to stay within range.")
                logging.info("Advice: HOLD - Price likely to stay within range.")

            # Sleep to avoid continuous fetching even after a successful fetch
            print(f"Sleeping for {sleep_interval} seconds before next check...")
            logging.info(f"Sleeping for {sleep_interval} seconds before next check...")
            sleep(sleep_interval)

        except Exception as e:
            print(f"Error occurred: {e}")
            logging.error(f"Error occurred: {e}")
            sleep(sleep_interval)

# Step 6: Configure settings
SLEEP_INTERVAL = 300  # Time in seconds between checks (5 minutes)

# Step 7: Start the real-time trading simulation
real_time_trading(SLEEP_INTERVAL)


Fetching BTC/USDT data from CoinGecko...
Data fetched successfully from CoinGecko.
Estimated Parameters: Theta=0.006086123425183998, Mu=74551.81614102633, Sigma=0.0025156299669670157
Adjusted VAH: 74601.81614102633, Adjusted VAL: 74501.81614102633
Calculating exit time with Mu: 74551.81614102633, VAL: 74501.81614102633, VAH: 74601.81614102633
Advice: SELL - Price is likely to break DOWN from the range.
Sleeping for 300 seconds before next check...
Fetching BTC/USDT data from CoinGecko...
Data fetched successfully from CoinGecko.
Estimated Parameters: Theta=0.006086123425183998, Mu=74551.81614102633, Sigma=0.0025156299669670157
Adjusted VAH: 74601.81614102633, Adjusted VAL: 74501.81614102633
Calculating exit time with Mu: 74551.81614102633, VAL: 74501.81614102633, VAH: 74601.81614102633
Advice: SELL - Price is likely to break DOWN from the range.
Sleeping for 300 seconds before next check...
Fetching BTC/USDT data from CoinGecko...
Data fetched successfully from CoinGecko.
Estimated Par